In [1]:
import codecs
import nltk
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
import gensim
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pickle

C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [26]:
#read preprocessed data from local
with open('./Pickle_Data/citation_with_context.pk', 'rb') as f:
    texts_with_context = pickle.load(f)
#    print(texts_with_context)
with open('./Pickle_Data/pre_citation_with_context.pk', 'rb') as f:
    pre_texts_with_context = pickle.load(f)
#    print(pre_texts_with_context)
with open('./Pickle_Data/citation.pk', 'rb') as f:
    texts = pickle.load(f)
#    print(texts)
with open('./Pickle_Data/polarities.pk', 'rb') as f:
    polarities = pickle.load(f)
#    print(polarities)
with open('./Pickle_Data/purposes.pk', 'rb') as f:
    purposes = pickle.load(f)
#    print(purposes)

citation_X = texts
citation_with_context_X = texts_with_context
pre_citation_with_context_X = pre_texts_with_context
polarity_Y = polarities
purpose_Y = purposes

citation_X = np.asarray(citation_X)
citation_with_context_X = np.asarray(citation_with_context_X)
pre_citation_with_context_X = np.asarray(pre_citation_with_context_X)
#print(citation_X)
print("------------Example of citations and its length:---------------")
print(len(citation_X))
print(citation_X[0])
print("------------Example of citations with contexts and its length:------------")
print(len(citation_with_context_X))
print(citation_with_context_X[0])
print("------------Example of citations with contexts and preprocessing and its length:------------")
print(len(pre_citation_with_context_X))
print(pre_citation_with_context_X[0])

------------Example of citations and its length:---------------
2191
These problems formulations are similar to those studied in  and ; , respectively
------------Example of citations with contexts and its length:------------
2191
The second instantiation finds the borders of phrases beginning and end and then pairs them in an optimal way into different phrases These problems formulations are similar to those studied in  and ; , respectively The experimental results presented using the SNoW based approach compare favorably with previously published results, both for NPs and SV phrases A s important, we present a few experiments that shed light on some of the issues involved in using learned predictors that interact to produce the desired inference
------------Example of citations with contexts and preprocessing and its length:------------
2191
the second instantiation finds the borders of phrases beginning and end and then pairs them in an optimal way into different phrases these probl

In [3]:
model = KeyedVectors.load('word2vec.model')
print("----------------INFO: word2vec model is loaded----------------")

----------------INFO: word2vec model is loaded----------------


In [4]:
print(model.most_similar('algorithms'))
print(len(model['algorithms']))
print(model.most_similar('cat'))
print(model.most_similar('analysis'))
#print(model['citation'])
print(model['cat'][1])
print(model['cats'][1])
print(model['CAT'][1])
print(model['Cat'][1])

C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('algorithm', 0.8449932336807251), ('algorithmic', 0.719566822052002), ('mathematical_algorithms', 0.6391038298606873), ('algorithmically', 0.6338590383529663), ('Algorithms', 0.6307376623153687), ('mathematical_algorithm', 0.623949408531189), ('mathematical_models', 0.6149172186851501), ('computational_algorithms', 0.6090747714042664), ('optimization_algorithms', 0.6071847081184387), ('proprietary_algorithms', 0.5949397683143616)]
300


[('cats', 0.8099379539489746), ('dog', 0.7609456777572632), ('kitten', 0.7464985251426697), ('feline', 0.7326233983039856), ('beagle', 0.7150583267211914), ('puppy', 0.7075453996658325), ('pup', 0.6934291124343872), ('pet', 0.6891531348228455), ('felines', 0.6755931377410889), ('chihuahua', 0.6709762215614319)]


[('analyzes', 0.7746602296829224), ('statistical_analysis', 0.6847716569900513), ('Analysis', 0.6558783054351807), ('econometric_analysis', 0.603839635848999), ('assessment', 0.5894891023635864), ('anaylsis', 0.5846796035766602), ('analyis', 0.5808142423629761), ('analytical', 0.5791622996330261), ('evaluation', 0.5724815726280212), ('analyzed', 0.5600607991218567)]
0.20410156
0.2578125
-0.111816406
0.05444336


In [6]:
# Calculate vector for a example sentence
example_sentence = "Hallo I'm Yide"
print("Calculate vector for a example sentence: " + example_sentence)
token_sen = nltk.word_tokenize(example_sentence)
print(token_sen)

sen_len = 0
a = np.zeros(300)
for token in token_sen:
    if token in model.wv.vocab:
        sen_len = sen_len + 1
        a = a + model[token]
        #print(model[token][0])
print(len(a))
a =a/sen_len
print(a)

Calculate vector for a example sentence: Hallo I'm Yide
['Hallo', 'I', "'m", 'Yide']
300
[ 8.68072510e-02 -2.07901001e-02  4.37622070e-02  6.29577637e-02
 -1.70898438e-02  7.50579834e-02 -1.89590454e-03 -6.03027344e-02
  7.88574219e-02  7.54699707e-02 -3.47709656e-03 -9.82990265e-02
 -5.16510010e-02 -1.14746094e-01 -1.58569336e-01  9.86938477e-02
  2.44140625e-04  2.09655762e-01  1.56250000e-02 -6.50634766e-02
 -7.50732422e-03  1.56127930e-01  1.97570801e-01 -1.11083984e-02
  4.80957031e-02 -2.22854614e-02 -1.56616211e-01 -1.13525391e-02
  1.87377930e-02 -8.88977051e-02  3.02734375e-02  6.90307617e-02
 -7.77587891e-02  1.61666870e-02  4.13208008e-02  3.45458984e-02
 -1.20220184e-02  1.31225586e-01  1.20193481e-01  7.70263672e-02
  1.41418457e-01 -1.24023438e-01  1.93847656e-01  5.81665039e-02
  4.71191406e-02  3.66210938e-03 -1.41204834e-01 -4.62646484e-02
 -8.43811035e-03  2.14947462e-02 -3.32031250e-02  2.08251953e-01
  1.80236816e-01  7.18078613e-02  8.58459473e-02  9.58557129e-02
 

C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [7]:
# Calculate for each citation (whiout contexts) a Vector, save those vectors in a List -> vector_citations_X
vector_citations_X = []
for sen in citation_X:
    #Todo: try to use other tokenize methods
    token_sen = nltk.word_tokenize(sen)
    sen_len = 0
    vec_sen = np.zeros(300)
    for token in token_sen:
        if token in model.wv.vocab:
            sen_len = sen_len + 1
            vec_sen = vec_sen + model[token]
    #print(len(vec_sen))
    vec_sen = vec_sen / sen_len
    vector_citations_X.append(vec_sen)
print("--------Number of citations converted to vectors: ------------")
print(len(vector_citations_X))
#print("-------Example vector for first citation: --------------------")
#print(vector_citations_X[0])

C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


--------Number of citations converted to vectors: ------------
2191


In [8]:
# Calculate for each citations (with contexts) a Vector, save those vectors in a List -> vector_citation_with_contexts_X
vector_citation_with_contexts_X = []
for sen in citation_with_context_X:
    token_sen = nltk.word_tokenize(sen)
    sen_len = 0
    vec_sen = np.zeros(300)
    for token in token_sen:
        if token in model.wv.vocab:
            sen_len = sen_len + 1
            vec_sen = vec_sen + model[token]
    #print(len(vec_sen))
    vec_sen = vec_sen / sen_len
    vector_citation_with_contexts_X.append(vec_sen)
print("--------Number of citations converted to vectors: ------------")
print(len(vector_citation_with_contexts_X))
#print("-------Example vector for first citation: --------------------")
#print(vector_citation_with_contexts_X[0])

C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


--------Number of citations converted to vectors: ------------
2191


In [27]:
# Calculate for each citations (with contexts and preprocessing) a Vector, save those vectors in a List -> vector_citation_with_contexts_X
vector_pre_citation_with_contexts_X = []
for sen in pre_citation_with_context_X:
    token_sen = nltk.word_tokenize(sen)
    sen_len = 0
    vec_sen = np.zeros(300)
    for token in token_sen:
        if token in model.wv.vocab:
            sen_len = sen_len + 1
            vec_sen = vec_sen + model[token]
    #print(len(vec_sen))
    vec_sen = vec_sen / sen_len
    vector_pre_citation_with_contexts_X.append(vec_sen)
print("--------Number of citations converted to vectors: ------------")
print(len(vector_citation_with_contexts_X))
#print("-------Example vector for first citation: --------------------")
#print(vector_citation_with_contexts_X[0])

C:\Users\songi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


--------Number of citations converted to vectors: ------------
2191


In [10]:
# Save vectors of each citations to local.
with open('./Pickle_Data/citation_vec.pk', 'wb') as f:
    pickle.dump(vector_citations_X, f)
# with open('C:/Users/songi/PycharmProjects/MasterThesis/citation_vec.pk', 'rb') as f:
#     data = pickle.load(f)
#     print(data[0])
print("--------Vectors saved in local------------")

--------Vectors saved in local------------


In [11]:
# Save vectors of each citations (with contexts) to local.
with open('./Pickle_Data/citation_with_context_vec.pk', 'wb') as f:
    pickle.dump(vector_citation_with_contexts_X, f)
# with open('C:/Users/songi/PycharmProjects/MasterThesis/citation_with_context_vec.pk', 'rb') as f:
#     data = pickle.load(f)
#     print(data[0])
print("--------Vectors saved in local------------")

--------Vectors saved in local------------


In [28]:
# Save vectors of each citations (with contexts) to local.
with open('./Pickle_Data/pre_citation_with_context_vec.pk', 'wb') as f:
    pickle.dump(vector_pre_citation_with_contexts_X, f)
# with open('C:/Users/songi/PycharmProjects/MasterThesis/citation_with_context_vec.pk', 'rb') as f:
#     data = pickle.load(f)
#     print(data[0])
print("--------Vectors saved in local------------")

--------Vectors saved in local------------
